# Time-Dependent Delivery Network

Design a time-dependent delivery network based on the specifications below.

## Problem Description:
Create a delivery network that schedules deliveries across multiple locations using a fleet of vehicles. The network must account for vehicle capacities, location storage capacities, delivery time windows, dynamic travel times, and vehicle speeds to ensure efficient and timely deliveries.

## Constraints:
1. **Locations:**
   - **Total Locations:** 15, labeled from `L0` to `L14`.
   - **Attributes:**
     - **Storage Capacity:** Each location has a storage capacity specified in kilograms (kg). Example: `L0` has a capacity of 500 kg.
     - **Time Window:** Each location has a delivery time window represented as a list of two integers `[start_hour, end_hour]` in 24-hour format. Example: `L3` has a time window of `[9, 11]` corresponding to 09:00-11:00.

2. **Vehicles:**
   - **Total Vehicles:** 7, labeled from `V1` to `V7`.
   - **Attributes:**
     - **Capacity:** Each vehicle has a specific capacity in kilograms (kg). Example: `V1` has a capacity of 100 kg.
     - **Speed:** Each vehicle has a defined speed in kilometers per hour (km/h). Example: `V1` travels at 60 km/h.

3. **Edges (Routes):**
   - **Definition:** Represents travel paths between two distinct locations.
   - **Attributes:**
     - **From:** The starting location ID (e.g., `L0`).
     - **To:** The destination location ID (e.g., `L1`).
     - **Base Travel Time:** The fundamental travel time for the route in minutes.
     - **Hourly Adjustments:** A dictionary where keys are time ranges in the format `"HH-HH"` (24-hour format) and values are additional travel time in minutes applicable during those hours. Example: `{"8-10": 15}` adds 15 minutes to the base travel time between 08:00-10:00.
     - **Maximum Weight Limit:** The maximum weight a vehicle can carry on that route in kilograms (kg).

4. **Operational Constraints:**
   - **Storage Capacity Compliance:** The sum of incoming goods to any location must not exceed its storage capacity.
   - **Vehicle Capacity Compliance:** No vehicle should exceed its capacity on any edge it traverses.
   - **Time Window Compliance:** Departures and arrivals must respect the time windows of locations. Specifically:
     - **Departure Time:** Must be within the `from` location's time window.
     - **Arrival Time:** Must be within the `to` location's time window.
     - **Loading Time:** Assume a fixed loading time of 10 minutes at each location, which must be accounted for when scheduling departures.

## Required Output Format:
Provide the network details in the following Python dictionary format. **Strictly adhere to this structure without deviations.**

```python
locations = [
    {"id": "L0", "capacity": int, "time_window": [int, int]},
    {"id": "L1", "capacity": int, "time_window": [int, int]},
    # ... all locations (L2-L14)
]

vehicles = [
    {"id": "V1", "capacity": int, "speed": int},
    {"id": "V2", "capacity": int, "speed": int},
    # ... all vehicles (V3-V7)
]

edges = [
    {"from": "L0", "to": "L1", "base_time": int, "hourly_adjust": {"HH-HH": int}, "max_weight": int},
    {"from": "L1", "to": "L2", "base_time": int, "hourly_adjust": {"HH-HH": int}, "max_weight": int},
    # ... more edges
]


In [1]:
from collections import defaultdict
import re

def verify_time_dependent_delivery_network(locations, vehicles, edges):
    verification_results = {
        "output_structure": {"passed": False, "message": ""},
        "storage_capacity_compliance": {"passed": False, "message": ""},
        "vehicle_capacity_compliance": {"passed": False, "message": ""},
        "time_window_compliance": {"passed": False, "message": ""},
        "overall_passed": False,
        "errors": []
    }
    def add_error(message):
        verification_results["errors"].append(message)

    if not isinstance(locations, list):
        add_error("'locations' should be a list.")
    elif len(locations) != 15:
        add_error(f"Number of locations is {len(locations)}; expected 15.")
    else:
        location_ids = set()
        location_valid = True
        for loc in locations:
            if not isinstance(loc, dict):
                add_error("Each location should be a dictionary.")
                location_valid = False
                continue
            required_loc_keys = {"id", "capacity", "time_window"}
            if not required_loc_keys.issubset(loc.keys()):
                add_error(f"Location {loc.get('id', 'Unknown')} is missing required keys: {required_loc_keys - set(loc.keys())}.")
                location_valid = False
                continue
            if not re.fullmatch(r"L\d+", loc["id"]):
                add_error(f"Location ID '{loc['id']}' is invalid. Expected format 'L0' to 'L14'.")
                location_valid = False
            location_ids.add(loc["id"])
            if not isinstance(loc["capacity"], int) or loc["capacity"] <= 0:
                add_error(f"Location '{loc['id']}' has invalid 'capacity'. Must be a positive integer.")
                location_valid = False
            if (not isinstance(loc["time_window"], list) or
                len(loc["time_window"]) != 2 or
                not all(isinstance(t, int) for t in loc["time_window"])):
                add_error(f"Location '{loc['id']}' has invalid 'time_window'. Must be a list of two integers.")
                location_valid = False
            else:
                start, end = loc["time_window"]
                if not (0 <= start < end <= 24):
                    add_error(f"Location '{loc['id']}' has invalid 'time_window' values: {loc['time_window']}. Must satisfy 0 <= start < end <= 24.")
                    location_valid = False
        if location_valid:
            verification_results["output_structure"]["passed"] = True
        else:
            verification_results["output_structure"]["message"] = "Errors found in 'locations' structure."
    
    if not isinstance(vehicles, list):
        add_error("'vehicles' should be a list.")
    elif len(vehicles) != 7:
        add_error(f"Number of vehicles is {len(vehicles)}; expected 7.")
    else:
        vehicle_ids = set()
        vehicle_valid = True
        vehicle_capacities = []
        for veh in vehicles:
            if not isinstance(veh, dict):
                add_error("Each vehicle should be a dictionary.")
                vehicle_valid = False
                continue
            required_veh_keys = {"id", "capacity", "speed"}
            if not required_veh_keys.issubset(veh.keys()):
                add_error(f"Vehicle {veh.get('id', 'Unknown')} is missing required keys: {required_veh_keys - set(veh.keys())}.")
                vehicle_valid = False
                continue
            if not re.fullmatch(r"V\d+", veh["id"]):
                add_error(f"Vehicle ID '{veh['id']}' is invalid. Expected format 'V1' to 'V7'.")
                vehicle_valid = False
            vehicle_ids.add(veh["id"])
            if not isinstance(veh["capacity"], int) or veh["capacity"] <= 0:
                add_error(f"Vehicle '{veh['id']}' has invalid 'capacity'. Must be a positive integer.")
                vehicle_valid = False
            else:
                vehicle_capacities.append(veh["capacity"])
            if not isinstance(veh["speed"], int) or veh["speed"] <= 0:
                add_error(f"Vehicle '{veh['id']}' has invalid 'speed'. Must be a positive integer.")
                vehicle_valid = False
        if vehicle_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'vehicles' structure."
    
    if not isinstance(edges, list):
        add_error("'edges' should be a list.")
    elif len(edges) == 0:
        add_error("No edges defined in the network.")
    else:
        edge_pairs = set()
        incoming_weights = defaultdict(int)
        edge_valid = True
        for edge in edges:
            if not isinstance(edge, dict):
                add_error("Each edge should be a dictionary.")
                edge_valid = False
                continue
            required_edge_keys = {"from", "to", "base_time", "hourly_adjust", "max_weight"}
            if not required_edge_keys.issubset(edge.keys()):
                add_error(f"Edge from '{edge.get('from', 'Unknown')}' to '{edge.get('to', 'Unknown')}' is missing required keys: {required_edge_keys - set(edge.keys())}.")
                edge_valid = False
                continue
            from_id = edge["from"]
            to_id = edge["to"]
            if from_id not in location_ids:
                add_error(f"Edge 'from' location '{from_id}' is not defined among locations.")
                edge_valid = False
            if to_id not in location_ids:
                add_error(f"Edge 'to' location '{to_id}' is not defined among locations.")
                edge_valid = False
            pair = (from_id, to_id)
            if pair in edge_pairs:
                add_error(f"Duplicate edge detected from '{from_id}' to '{to_id}'.")
                edge_valid = False
            else:
                edge_pairs.add(pair)
            if not isinstance(edge["base_time"], int) or edge["base_time"] <= 0:
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'base_time'. Must be a positive integer.")
                edge_valid = False
            if not isinstance(edge["hourly_adjust"], dict):
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'hourly_adjust'. Must be a dictionary.")
                edge_valid = False
            else:
                for time_range, adjustment in edge["hourly_adjust"].items():
                    if not re.fullmatch(r"\d{1,2}-\d{1,2}", time_range):
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid time range '{time_range}' in 'hourly_adjust'. Expected format 'HH-HH'.")
                        edge_valid = False
                        continue
                    start, end = map(int, time_range.split('-'))
                    if not (0 <= start < end <= 24):
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid time range values '{time_range}'. Must satisfy 0 <= start < end <= 24.")
                        edge_valid = False
                    if not isinstance(adjustment, int) or adjustment < 0:
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid adjustment value '{adjustment}' for time range '{time_range}'. Must be a non-negative integer.")
                        edge_valid = False
            if not isinstance(edge["max_weight"], int) or edge["max_weight"] <= 0:
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'max_weight'. Must be a positive integer.")
                edge_valid = False
            else:
                incoming_weights[to_id] += edge["max_weight"]
        if edge_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'edges' structure."
    

    storage_compliance_passed = True
    storage_errors = []
    loc_capacity_map = {loc["id"]: loc["capacity"] for loc in locations}
    for loc_id, total_incoming in incoming_weights.items():
        capacity = loc_capacity_map.get(loc_id, 0)
        if total_incoming > capacity:
            storage_compliance_passed = False
            storage_errors.append(f"Total incoming 'max_weight' to location '{loc_id}' is {total_incoming} kg, exceeding its storage capacity of {capacity} kg.")
    if storage_compliance_passed:
        verification_results["storage_capacity_compliance"]["passed"] = True
    else:
        verification_results["storage_capacity_compliance"]["message"] = " ".join(storage_errors)
        add_error("Storage Capacity Compliance Check Failed.")
    
    vehicle_capacity_passed = True
    vehicle_errors = []
    if not vehicle_capacities:
        vehicle_capacity_passed = False
        vehicle_errors.append("No valid vehicle capacities found.")
    else:
        max_vehicle_capacity = max(vehicle_capacities)
        for edge in edges:
            if edge["max_weight"] < max_vehicle_capacity:
                vehicle_capacity_passed = False
                vehicle_errors.append(f"Edge from '{edge['from']}' to '{edge['to']}' has 'max_weight' {edge['max_weight']} kg, which is less than the maximum vehicle capacity of {max_vehicle_capacity} kg.")
    if vehicle_capacity_passed:
        verification_results["vehicle_capacity_compliance"]["passed"] = True
    else:
        verification_results["vehicle_capacity_compliance"]["message"] = " ".join(vehicle_errors)
        add_error("Vehicle Capacity Compliance Check Failed.")
    
    time_window_passed = True
    time_window_errors = []
    loc_time_map = {loc["id"]: loc["time_window"] for loc in locations}
    for edge in edges:
        from_id = edge["from"]
        to_id = edge["to"]
        base_time = edge["base_time"]  
        hourly_adjust = edge["hourly_adjust"] 
        max_adjust = max(hourly_adjust.values()) if hourly_adjust else 0
        total_travel_time = base_time + max_adjust  # in minutes
        total_travel_hours = (total_travel_time + 59) // 60 
        from_tw_start, from_tw_end = loc_time_map[from_id]
        to_tw_start, to_tw_end = loc_time_map[to_id]
        feasible = False
        for dep_hour in range(from_tw_start, from_tw_end):
            dep_time = dep_hour + 0.17 
            arr_time = dep_time + (total_travel_time / 60)
            if to_tw_start <= arr_time <= to_tw_end:
                feasible = True
                break
        if not feasible:
            time_window_passed = False
            time_window_errors.append(f"Edge from '{from_id}' to '{to_id}' with total travel time {total_travel_time} minutes does not fit within the time windows of '{from_id}' [{from_tw_start}-{from_tw_end}] and '{to_id}' [{to_tw_start}-{to_tw_end}].")
    if time_window_passed:
        verification_results["time_window_compliance"]["passed"] = True
    else:
        verification_results["time_window_compliance"]["message"] = " ".join(time_window_errors)
        add_error("Time Window Compliance Check Failed.")
    
    overall_passed = all([
        verification_results["output_structure"]["passed"],
        verification_results["storage_capacity_compliance"]["passed"],
        verification_results["vehicle_capacity_compliance"]["passed"],
        verification_results["time_window_compliance"]["passed"]
    ])
    verification_results["overall_passed"] = overall_passed
    
    return overall_passed, verification_results

#PASTE HERE!

# Execute verification
verification_output = verify_time_dependent_delivery_network(locations, vehicles, edges)
print("Verification Output:", verification_output[1])


NameError: name 'locations' is not defined

In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o1mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "timeDependentDeliveryNetwork"


def get_user_input():
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_run{run_number}_output.json")


verification_json = {
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


# Directed Social Network with Influence Relationships

Design a directed social network based on the specifications below.

## Problem Description:
Create a social network graph representing influence relationships among users. Each user has specific attributes, and influence connections must adhere to defined constraints to maintain the integrity and intended structure of the network.

## Constraints:
1. **Users:**
   - Total of 20 users labeled from U0 to U19.
   - Each user has a "trust_score" ranging from 0 to 100.
   - Each user belongs to a "category" based on their trust score:
     - "celebrity" (trust_score ≥ 80)
     - "expert" (70 ≤ trust_score < 80)
     - "regular" (trust_score < 50)

2. **Edges (Influence Relationships):**
   - Directed edges where Ux → Uy indicates that Ux influences Uy.
   - **No self-loops:** A user cannot influence themselves.
   - **Category Constraints:**
     - **Celebrities:** Must have at least 5 outgoing edges.
     - **Regular Users:** Cannot influence experts.
   - **Graph Structure:**
     - The graph must be acyclic (no cycles in influence relationships).

### Required Output Format:
Provide the network details in the following Python dictionary format. **Strictly adhere to this structure without deviations.**


users = [
    {"id": "U0", "trust_score": int, "category": "celebrity"},
    {"id": "U1", "trust_score": int, "category": "regular"},
    {"id": "U2", "trust_score": int, "category": "expert"},
    # ... all users (U3-U19)
]

edges = [
    {"from": "U0", "to": "U1"},
    {"from: "U0", "to": "U2"},
    {"from": "U2", "to": "U3"},
    # ... more edges
]


In [ ]:
from collections import defaultdict  

def verify_social_network(users, edges):

    validation_results = {
        "valid_no_self_loops": True,
        "valid_celebrity_outgoing": True,
        "valid_regular_to_expert": True,
        "valid_acyclic": True,
        "valid_user_attributes": True,
        "valid_edge_structure": True,
        "valid_total_users": True,
        "valid_categories": True,
        "valid_trust_scores": True,
        "errors": []
    }

    if not isinstance(users, list):
        validation_results["valid_total_users"] = False
        validation_results["errors"].append("'users' should be a list.")
    elif len(users) != 20:
        validation_results["valid_total_users"] = False
        validation_results["errors"].append(f"Number of users is {len(users)}; expected 20.")
    
    expected_user_ids = {f"U{i}" for i in range(20)}
    actual_user_ids = set()
    user_map = {}
    for user in users:
        if not isinstance(user, dict):
            validation_results["valid_user_attributes"] = False
            validation_results["errors"].append("Each user should be a dictionary.")
            continue
        required_keys = {"id", "trust_score", "category"}
        if not required_keys.issubset(user.keys()):
            missing = required_keys - user.keys()
            validation_results["valid_user_attributes"] = False
            validation_results["errors"].append(f"User {user.get('id', 'Unknown')} is missing keys: {missing}.")
            continue
        user_id = user["id"]
        if user_id not in expected_user_ids:
            validation_results["valid_user_attributes"] = False
            validation_results["errors"].append(f"User ID '{user_id}' is invalid. Expected IDs from U0 to U19.")
        actual_user_ids.add(user_id)
        user_map[user_id] = user
        trust_score = user["trust_score"]
        if not isinstance(trust_score, int) or not (0 <= trust_score <= 100):
            validation_results["valid_trust_scores"] = False
            validation_results["errors"].append(f"User '{user_id}' has invalid 'trust_score': {trust_score}. Must be an integer between 0 and 100.")
        category = user["category"]
        if trust_score >= 80 and category != "celebrity":
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'celebrity'.")
        elif 70 <= trust_score < 80 and category != "expert":
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'expert'.")
        elif trust_score < 50 and category != "regular":
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'regular'.")
        elif 50 <= trust_score < 70 and category not in ["expert", "regular"]:
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'expert' or 'regular'.")

    missing_users = expected_user_ids - actual_user_ids
    if missing_users:
        validation_results["valid_total_users"] = False
        validation_results["errors"].append(f"Missing user IDs: {missing_users}.")

    if not isinstance(edges, list):
        validation_results["valid_edge_structure"] = False
        validation_results["errors"].append("'edges' should be a list.")
    else:
        adj = defaultdict(list)
        for edge in edges:
            if not isinstance(edge, dict):
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append("Each edge should be a dictionary.")
                continue
            required_edge_keys = {"from", "to"}
            if not required_edge_keys.issubset(edge.keys()):
                missing = required_edge_keys - edge.keys()
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append(f"Edge is missing keys: {missing}.")
                continue
            from_id = edge["from"]
            to_id = edge["to"]
            if from_id not in expected_user_ids:
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append(f"Edge 'from' user '{from_id}' is not a valid user ID.")
            if to_id not in expected_user_ids:
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append(f"Edge 'to' user '{to_id}' is not a valid user ID.")
            adj[from_id].append(to_id)

    for edge in edges:
        if edge["from"] == edge["to"]:
            validation_results["valid_no_self_loops"] = False
            validation_results["errors"].append(f"Self-loop detected: {edge['from']} → {edge['to']}.")

    for user in users:
        if user["category"] == "celebrity":
            user_id = user["id"]
            outgoing = len(adj[user_id])
            if outgoing < 5:
                validation_results["valid_celebrity_outgoing"] = False
                validation_results["errors"].append(f"Celebrity '{user_id}' has {outgoing} outgoing edges; expected at least 5.")

    for edge in edges:
        from_user = user_map.get(edge["from"])
        to_user = user_map.get(edge["to"])
        if from_user and to_user:
            if from_user["category"] == "regular" and to_user["trust_score"] >= 70:
                validation_results["valid_regular_to_expert"] = False
                validation_results["errors"].append(f"Regular user '{from_user['id']}' is influencing expert '{to_user['id']}'.")

    visited = set()
    rec_stack = set()
    def dfs(u):
        visited.add(u)
        rec_stack.add(u)
        for v in adj[u]:
            if v not in visited:
                if dfs(v):
                    return True
            elif v in rec_stack:
                return True
        rec_stack.remove(u)
        return False

    has_cycle = False
    for user_id in expected_user_ids:
        if user_id not in visited:
            if dfs(user_id):
                has_cycle = True
                break
    if has_cycle:
        validation_results["valid_acyclic"] = False
        validation_results["errors"].append("Cycle detected in the network; the graph must be acyclic.")
    return(validation_results['valid_no_self_loops'] and
    validation_results['valid_celebrity_outgoing'] and 
    validation_results['valid_regular_to_expert'] and
     validation_results['valid_acyclic'] and
     validation_results['valid_user_attributes'] and
     validation_results['valid_edge_structure'] and
     validation_results['valid_total_users'] and
      validation_results['valid_categories'] and
      validation_results['valid_trust_scores']
      , validation_results)


#PASTE HERE!

verification_output = verify_social_network(users,edges)
print("Verification Output:", verification_output)



In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o1mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "directedSocialNetwork"


def get_user_input():
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_run{run_number}_output.json")


verification_json = {
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)



# Quantum Circuit Design

Design a quantum circuit based on the specifications below.

## Problem Description:
Create a quantum circuit consisting of multiple qubits and quantum gates. The circuit must adhere to specific constraints to ensure proper gate operations and overall circuit functionality.

## Constraints:
1. **Qubits:**
   - Total of 15 qubits labeled from Q0 to Q14.

2. **Gates:**
   - Types of gates to include: 
     - Hadamard (H)
     - Controlled NOT (CNOT)
     - T-gate (T)
     - Measure (Measure)
   - **Gate Operations:**
     - Each gate operates on specific qubits at designated times.
     - **T-gates:** Must be preceded by an H-gate on the same qubit.
     - **CNOT Gates:** Must operate on adjacent qubits (e.g., Q0 and Q1, Q1 and Q2).
     - **Measurements:** Each qubit can be measured only once.
   
3. **Circuit Structure:**
   - The circuit must be a Directed Acyclic Graph (DAG); no repeated times for the same qubit.

## Required Output Format:

Provide the network details in the following Python dictionary format. **Strictly adhere to this structure without deviations.**

```python
circuit = [
    {"gate": "H", "qubits": ["Q0"], "time": 1},
    {"gate": "CNOT", "qubits": ["Q0", "Q1"], "time": 2},
    {"gate": "T", "qubits": ["Q0"], "time": 3},
    {"gate": "Measure", "qubits": ["Q0"], "time": 4},
    # ... more gates
]


In [ ]:
from collections import defaultdict

def verify_quantum_circuit(circuit):

    validation_results = {
        "valid_dag": True,
        "valid_tgates": True,
        "valid_measure": True,
        "valid_cnot_adjacency": True,
        "valid_output_structure": True,
        "valid_gate_types": True,
        "valid_qubits": True,
        "errors": []
    }


    if not isinstance(circuit, list):
        validation_results["valid_output_structure"] = False
        validation_results["errors"].append("'circuit' should be a list.")
        return validation_results  # Cannot proceed further

    qubit_timeline = {f"Q{i}": [] for i in range(15)}
    valid_qubits = set(qubit_timeline.keys())

    allowed_gates = {"H", "CNOT", "T", "Measure"}

    cnot_gates = []

    measurement_operations = defaultdict(int)

    for idx, op in enumerate(circuit):
        if not isinstance(op, dict):
            validation_results["valid_output_structure"] = False
            validation_results["errors"].append(f"Gate operation at index {idx} is not a dictionary.")
            continue

        required_keys = {"gate", "qubits", "time"}
        if not required_keys.issubset(op.keys()):
            missing = required_keys - op.keys()
            validation_results["valid_output_structure"] = False
            validation_results["errors"].append(f"Gate operation at index {idx} is missing keys: {missing}.")
            continue

        gate = op["gate"]
        qubits = op["qubits"]
        time = op["time"]

        if gate not in allowed_gates:
            validation_results["valid_gate_types"] = False
            validation_results["errors"].append(f"Invalid gate type '{gate}' at index {idx}. Allowed gates: {allowed_gates}.")
        
        if not isinstance(qubits, list) or not all(isinstance(q, str) for q in qubits):
            validation_results["valid_output_structure"] = False
            validation_results["errors"].append(f"'qubits' for gate at index {idx} must be a list of strings.")
            continue

        for q in qubits:
            if q not in valid_qubits:
                validation_results["valid_qubits"] = False
                validation_results["errors"].append(f"Invalid qubit ID '{q}' in gate at index {idx}. Expected IDs from Q0 to Q14.")

        if not isinstance(time, int) or time < 0:
            validation_results["valid_output_structure"] = False
            validation_results["errors"].append(f"Invalid 'time' value '{time}' in gate at index {idx}. Must be a non-negative integer.")
            continue

        for q in qubits:
            qubit_timeline[q].append((time, gate))

        if gate == "CNOT":
            if len(qubits) != 2:
                validation_results["valid_output_structure"] = False
                validation_results["errors"].append(f"CNOT gate at index {idx} must operate on exactly two qubits.")
            else:
                cnot_gates.append(op)

        if gate == "Measure":
            measurement_operations[qubits[0]] += 1

    for q, ops in qubit_timeline.items():
        times = [t for (t, _) in ops]
        if len(times) != len(set(times)):
            validation_results["valid_dag"] = False
            validation_results["errors"].append(f"Qubit '{q}' has overlapping operations at the same time.")
    
    for q, ops in qubit_timeline.items():
        sorted_ops = sorted(ops, key=lambda x: x[0])
        has_prev_h = False
        for time, gate in sorted_ops:
            if gate == "H":
                has_prev_h = True
            elif gate == "T":
                if not has_prev_h:
                    validation_results["valid_tgates"] = False
                    validation_results["errors"].append(f"T-gate on qubit '{q}' at time {time} is not preceded by an H-gate.")
                has_prev_h = False  
            elif gate == "Measure":
                has_prev_h = False 

    for q, count in measurement_operations.items():
        if count > 1:
            validation_results["valid_measure"] = False
            validation_results["errors"].append(f"Qubit '{q}' has been measured {count} times; only one measurement allowed.")

    for idx, cnot in enumerate(cnot_gates):
        ctrl = cnot["qubits"][0]
        target = cnot["qubits"][1]
        ctrl_num = int(ctrl[1:])
        target_num = int(target[1:])
        if abs(ctrl_num - target_num) != 1:
            validation_results["valid_cnot_adjacency"] = False
            validation_results["errors"].append(f"CNOT gate at index {idx} operates on non-adjacent qubits '{ctrl}' and '{target}'.")

    return(validation_results['valid_dag'] and
    validation_results['valid_tgates'] and 
    validation_results['valid_measure'] and
     validation_results['valid_cnot_adjacency'] and
     validation_results['valid_output_structure'] and
     validation_results['valid_gate_types'] and
     validation_results['valid_qubits'], validation_results)


# PASTE HERE!

verification_output = verify_quantum_circuit(circuit)
verification_output

In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o1mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "quantum"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_run{run_number}_output.json")


verification_json = {
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)

# Gene-Disease Association Network

Design a gene-disease association network based on the specifications below.

## Problem Description:
Create a bipartite network that models the associations between genes and diseases. This network will represent which genes are associated with which diseases, capturing the strength of each association. The network should adhere to defined constraints to ensure biological relevance and structural integrity.

## Constraints:
1. **Nodes:**
   - **Genes:**
     - Total of 20 genes labeled from G0 to G19.
     - Each gene has a "name" and a "function".
   - **Diseases:**
     - Total of 20 diseases labeled from D0 to D19.
     - Each disease has a "name" and a "severity_level" (e.g., "Low", "Medium", "High").
   
2. **Edges (Associations):**
   - Represents the association between a gene and a disease.
   - **Bipartite Constraint:** Associations can only exist between genes and diseases, not within the same set.
   - **Association Strength:** Each association has a "strength" value ranging from 0.0 to 1.0, indicating the confidence of the association.
   
3. **Degree Constraints:**
   - **Genes:**
     - Each gene must be associated with at least 2 and at most 5 diseases.
   - **Diseases:**
     - Each disease must be associated with at least 3 and at most 10 genes.
   
4. **Structural Constraints:**
   - The network must be bipartite; no edges should connect nodes within the same set (i.e., no gene-gene or disease-disease associations).
   - There should be no duplicate edges (i.e., each gene-disease pair is unique).

## Required Output Format:

Provide the network details in the following Python dictionary format. **Strictly adhere to this structure without deviations.**

```python
genes = [
    {"id": "G0", "name": "BRCA1", "function": "DNA repair"},
    {"id": "G1", "name": "TP53", "function": "Tumor suppression"},
    # ... all genes (G2-G19)
]

diseases = [
    {"id": "D0", "name": "Breast Cancer", "severity_level": "High"},
    {"id": "D1", "name": "Lung Cancer", "severity_level": "High"},
    # ... all diseases (D2-D19)
]

associations = [
    {"from": "G0", "to": "D0", "strength": 0.85},
    {"from": "G0", "to": "D2", "strength": 0.60},
    {"from": "G1", "to": "D0", "strength": 0.90},
    # ... more associations
]


In [ ]:
def verify_gene_disease_association_network(genes, diseases, associations):
    verification_results = {
        "defined_counts": {"passed": False, "message": ""},
        "valid_associations": {"passed": False, "message": ""},
        "degree_constraints": {"passed": False, "message": ""},
        "duplicate_associations": {"passed": False, "message": ""},
    }

    error_messages = []

    if len(genes) == 20 and len(diseases) == 20:
        verification_results["defined_counts"]["passed"] = True
    else:
        missing = []
        if len(genes) != 20:
            missing.append(f"genes count is {len(genes)}; expected 20.")
        if len(diseases) != 20:
            missing.append(f"diseases count is {len(diseases)}; expected 20.")
        verification_results["defined_counts"]["message"] = " ".join(missing)
        error_messages.append("Defined Counts Check Failed.")

    gene_map = {gene["id"]: gene for gene in genes}
    disease_map = {disease["id"]: disease for disease in diseases}

    invalid_associations = []
    for assoc in associations:
        from_id = assoc.get("from")
        to_id = assoc.get("to")
        strength = assoc.get("strength")

        if from_id not in gene_map:
            invalid_associations.append(f"Association from undefined gene '{from_id}' to '{to_id}'.")
        
        if to_id not in disease_map:
            invalid_associations.append(f"Association from '{from_id}' to undefined disease '{to_id}'.")
        
        if not isinstance(strength, (int, float)) or not (0.0 <= strength <= 1.0):
            invalid_associations.append(f"Association strength {strength} for '{from_id}' -> '{to_id}' is out of bounds (0.0 - 1.0).")
    
    if not invalid_associations:
        verification_results["valid_associations"]["passed"] = True
    else:
        verification_results["valid_associations"]["message"] = " ".join(invalid_associations)
        error_messages.append("Valid Associations Check Failed.")

    gene_degrees = defaultdict(int)
    disease_degrees = defaultdict(int)
    for assoc in associations:
        gene_id = assoc.get("from")
        disease_id = assoc.get("to")
        gene_degrees[gene_id] += 1
        disease_degrees[disease_id] += 1

    degree_errors = []

    for gene in genes:
        gene_id = gene["id"]
        degree = gene_degrees.get(gene_id, 0)
        if degree < 2 or degree > 5:
            degree_errors.append(f"Gene '{gene_id}' has {degree} associations; expected between 2 and 5.")

    for disease in diseases:
        disease_id = disease["id"]
        degree = disease_degrees.get(disease_id, 0)
        if degree < 3 or degree > 10:
            degree_errors.append(f"Disease '{disease_id}' has {degree} associations; expected between 3 and 10.")

    if not degree_errors:
        verification_results["degree_constraints"]["passed"] = True
    else:
        verification_results["degree_constraints"]["message"] = " ".join(degree_errors)
        error_messages.append("Degree Constraints Check Failed.")

    seen_pairs = set()
    duplicates = []
    for assoc in associations:
        pair = (assoc.get("from"), assoc.get("to"))
        if pair in seen_pairs:
            duplicates.append(f"Duplicate association found between '{pair[0]}' and '{pair[1]}'.")
        else:
            seen_pairs.add(pair)
    
    if not duplicates:
        verification_results["duplicate_associations"]["passed"] = True
    else:
        verification_results["duplicate_associations"]["message"] = " ".join(duplicates)
        error_messages.append("Duplicate Associations Check Failed.")

    final_results = {
        "constraints": verification_results,
        "overall_passed": not error_messages,
        "errors": error_messages
    }

    return (not error_messages, final_results)

# PASTE HERE!


verification_output = verify_gene_disease_association_network(genes, diseases, associations)
print("Gene-Disease Association Network is valid:", verification_output)



In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o1mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "geneAssociation"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_run{run_number}_output.json")


verification_json = {
    "result": is_valid[0],
    "fullOutput": is_valid[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)

# Electric Power Distribution Network

Design an electric power distribution network based on the specifications below.

## Problem Description:
Create a distribution network that transmits electricity from power plants to consumers through substations. The network should account for power plant capacities, substation storage capacities, consumer demands, transmission line capacities, dynamic transmission times, and operational constraints to ensure efficient and reliable power delivery.

## Constraints:
1. **Nodes:**
   - **Power Plants:**
     - **Total:** 5, labeled from `P0` to `P4`.
     - **Attributes:**
       - **Capacity:** Maximum power generation capacity in megawatts (MW).
   - **Substations:**
     - **Total:** 10, labeled from `S0` to `S9`.
     - **Attributes:**
       - **Storage Capacity:** Maximum power storage capacity in megawatt-hours (MWh).
   - **Consumers:**
     - **Total:** 20, labeled from `C0` to `C19`.
     - **Attributes:**
       - **Demand:** Power demand in megawatts (MW).
   
2. **Transmission Lines (Edges):**
   - **Definition:** Represents power transmission paths between two nodes (power plants, substations, consumers).
   - **Attributes:**
     - **From:** Starting node ID (e.g., `P0`, `S3`).
     - **To:** Destination node ID (e.g., `S1`, `C5`).
     - **Base Transmission Time:** Fundamental transmission time in minutes.
     - **Dynamic Transmission Time Adjustments:** A dictionary where keys are time ranges in the format `"HH-HH"` (24-hour format) and values are additional transmission time in minutes applicable during those hours. Example: `{"8-10": 15}` adds 15 minutes to the base transmission time between 08:00-10:00.
     - **Maximum Capacity:** Maximum power capacity of the transmission line in megawatts (MW).
   
3. **Operational Constraints:**
   - **Storage Capacity Compliance:** The sum of incoming power to any substation must not exceed its storage capacity.
   - **Transmission Capacity Compliance:** No transmission line should carry more power than its maximum capacity.
   - **Demand Fulfillment:** All consumer demands must be met without exceeding the capacities of power plants and transmission lines.
   - **Transmission Time Compliance:** Power transmission must respect dynamic transmission times based on the time of day.

## Required Output Format:
Provide the network details in the following Python dictionary format. **Strictly adhere to this structure without deviations.**

```python
power_plants = [
    {"id": "P0", "capacity": int},
    {"id": "P1", "capacity": int},
    # ... all power plants (P2-P4)
]

substations = [
    {"id": "S0", "capacity": int},
    {"id": "S1", "capacity": int},
    # ... all substations (S2-S9)
]

consumers = [
    {"id": "C0", "demand": int},
    {"id": "C1", "demand": int},
    # ... all consumers (C2-C19)
]

transmission_lines = [
    {"from": "P0", "to": "S0", "base_time": int, "dynamic_adjust": {"HH-HH": int}, "max_capacity": int},
    {"from": "S0", "to": "C0", "base_time": int, "dynamic_adjust": {"HH-HH": int}, "max_capacity": int},
    # ... more transmission lines
]


In [ ]:


from collections import defaultdict
import re

def verify_electric_power_distribution_network(power_plants, substations, consumers, transmission_lines):
    verification_results = {
        "output_structure": {"passed": False, "message": ""},
        "storage_capacity_compliance": {"passed": False, "message": ""},
        "transmission_capacity_compliance": {"passed": False, "message": ""},
        "demand_fulfillment": {"passed": False, "message": ""},
        "transmission_time_compliance": {"passed": False, "message": ""},
        "overall_passed": False,
        "errors": []
    }

    def add_error(message):
        verification_results["errors"].append(message)
    if not isinstance(power_plants, list):
        add_error("'power_plants' should be a list.")
    elif len(power_plants) != 5:
        add_error(f"Number of power plants is {len(power_plants)}; expected 5.")
    else:
        power_plant_ids = set()
        power_plant_valid = True
        for plant in power_plants:
            if not isinstance(plant, dict):
                add_error("Each power plant should be a dictionary.")
                power_plant_valid = False
                continue
            required_plant_keys = {"id", "capacity"}
            if not required_plant_keys.issubset(plant.keys()):
                add_error(f"Power Plant {plant.get('id', 'Unknown')} is missing required keys: {required_plant_keys - set(plant.keys())}.")
                power_plant_valid = False
                continue
            if not re.fullmatch(r"P\d+", plant["id"]):
                add_error(f"Power Plant ID '{plant['id']}' is invalid. Expected format 'P0' to 'P4'.")
                power_plant_valid = False
            power_plant_ids.add(plant["id"])
            if not isinstance(plant["capacity"], int) or plant["capacity"] <= 0:
                add_error(f"Power Plant '{plant['id']}' has invalid 'capacity'. Must be a positive integer.")
                power_plant_valid = False
        if power_plant_valid:
            verification_results["output_structure"]["passed"] = True
        else:
            verification_results["output_structure"]["message"] = "Errors found in 'power_plants' structure."

    if not isinstance(substations, list):
        add_error("'substations' should be a list.")
    elif len(substations) != 10:
        add_error(f"Number of substations is {len(substations)}; expected 10.")
    else:
        substation_ids = set()
        substation_valid = True
        for sub in substations:
            if not isinstance(sub, dict):
                add_error("Each substation should be a dictionary.")
                substation_valid = False
                continue
            required_sub_keys = {"id", "capacity"}
            if not required_sub_keys.issubset(sub.keys()):
                add_error(f"Substation {sub.get('id', 'Unknown')} is missing required keys: {required_sub_keys - set(sub.keys())}.")
                substation_valid = False
                continue
            if not re.fullmatch(r"S\d+", sub["id"]):
                add_error(f"Substation ID '{sub['id']}' is invalid. Expected format 'S0' to 'S9'.")
                substation_valid = False
            substation_ids.add(sub["id"])
            if not isinstance(sub["capacity"], int) or sub["capacity"] <= 0:
                add_error(f"Substation '{sub['id']}' has invalid 'capacity'. Must be a positive integer.")
                substation_valid = False
        if substation_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'substations' structure."

    if not isinstance(consumers, list):
        add_error("'consumers' should be a list.")
    elif len(consumers) != 20:
        add_error(f"Number of consumers is {len(consumers)}; expected 20.")
    else:
        consumer_ids = set()
        consumer_valid = True
        for consumer in consumers:
            if not isinstance(consumer, dict):
                add_error("Each consumer should be a dictionary.")
                consumer_valid = False
                continue
            required_consumer_keys = {"id", "demand"}
            if not required_consumer_keys.issubset(consumer.keys()):
                add_error(f"Consumer {consumer.get('id', 'Unknown')} is missing required keys: {required_consumer_keys - set(consumer.keys())}.")
                consumer_valid = False
                continue
            if not re.fullmatch(r"C\d+", consumer["id"]):
                add_error(f"Consumer ID '{consumer['id']}' is invalid. Expected format 'C0' to 'C19'.")
                consumer_valid = False
            consumer_ids.add(consumer["id"])
            if not isinstance(consumer["demand"], int) or consumer["demand"] <= 0:
                add_error(f"Consumer '{consumer['id']}' has invalid 'demand'. Must be a positive integer.")
                consumer_valid = False
        if consumer_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'consumers' structure."

    # d. Verify 'transmission_lines' list
    if not isinstance(transmission_lines, list):
        add_error("'transmission_lines' should be a list.")
    elif len(transmission_lines) == 0:
        add_error("No transmission lines defined in the network.")
    else:
        transmission_valid = True
        transmission_pairs = set()
        incoming_power = defaultdict(int)
        for line in transmission_lines:
            if not isinstance(line, dict):
                add_error("Each transmission line should be a dictionary.")
                transmission_valid = False
                continue
            required_line_keys = {"from", "to", "base_time", "dynamic_adjust", "max_capacity"}
            if not required_line_keys.issubset(line.keys()):
                add_error(f"Transmission Line from '{line.get('from', 'Unknown')}' to '{line.get('to', 'Unknown')}' is missing required keys: {required_line_keys - set(line.keys())}.")
                transmission_valid = False
                continue
            from_id = line["from"]
            to_id = line["to"]
            if from_id not in power_plant_ids and from_id not in substation_ids:
                add_error(f"Transmission Line 'from' node '{from_id}' is not defined among power plants or substations.")
                transmission_valid = False
            if to_id not in substation_ids and to_id not in consumer_ids:
                add_error(f"Transmission Line 'to' node '{to_id}' is not defined among substations or consumers.")
                transmission_valid = False
            pair = (from_id, to_id)
            if pair in transmission_pairs:
                add_error(f"Duplicate transmission line detected from '{from_id}' to '{to_id}'.")
                transmission_valid = False
            else:
                transmission_pairs.add(pair)
            if not isinstance(line["base_time"], int) or line["base_time"] <= 0:
                add_error(f"Transmission Line from '{from_id}' to '{to_id}' has invalid 'base_time'. Must be a positive integer.")
                transmission_valid = False
            if not isinstance(line["dynamic_adjust"], dict):
                add_error(f"Transmission Line from '{from_id}' to '{to_id}' has invalid 'dynamic_adjust'. Must be a dictionary.")
                transmission_valid = False
            else:
                for time_range, adjustment in line["dynamic_adjust"].items():
                    if not re.fullmatch(r"\d{1,2}-\d{1,2}", time_range):
                        add_error(f"Transmission Line from '{from_id}' to '{to_id}' has invalid time range '{time_range}' in 'dynamic_adjust'. Expected format 'HH-HH'.")
                        transmission_valid = False
                        continue
                    start, end = map(int, time_range.split('-'))
                    if not (0 <= start < end <= 24):
                        add_error(f"Transmission Line from '{from_id}' to '{to_id}' has invalid time range values '{time_range}'. Must satisfy 0 <= start < end <= 24.")
                        transmission_valid = False
                    if not isinstance(adjustment, int) or adjustment < 0:
                        add_error(f"Transmission Line from '{from_id}' to '{to_id}' has invalid adjustment value '{adjustment}' for time range '{time_range}'. Must be a non-negative integer.")
                        transmission_valid = False
            if not isinstance(line["max_capacity"], int) or line["max_capacity"] <= 0:
                add_error(f"Transmission Line from '{from_id}' to '{to_id}' has invalid 'max_capacity'. Must be a positive integer.")
                transmission_valid = False
            else:
                incoming_power[to_id] += line["max_capacity"]
        if transmission_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'transmission_lines' structure."


    storage_compliance_passed = True
    storage_errors = []
    substation_capacity_map = {sub["id"]: sub["capacity"] for sub in substations}
    for sub_id, total_incoming in incoming_power.items():
        if sub_id in substation_capacity_map:
            capacity = substation_capacity_map[sub_id]
            if total_incoming > capacity:
                storage_compliance_passed = False
                storage_errors.append(f"Total incoming 'max_capacity' to substation '{sub_id}' is {total_incoming} MW, exceeding its storage capacity of {capacity} MWh.")
    if storage_compliance_passed:
        verification_results["storage_capacity_compliance"]["passed"] = True
    else:
        verification_results["storage_capacity_compliance"]["message"] = " ".join(storage_errors)
        add_error("Storage Capacity Compliance Check Failed.")


    
    transmission_capacity_passed = True
    transmission_capacity_errors = []
    
    total_power_generated = sum(plant["capacity"] for plant in power_plants)
    total_power_demand = sum(consumer["demand"] for consumer in consumers)
    
    if total_power_generated < total_power_demand:
        transmission_capacity_passed = False
        transmission_capacity_errors.append(f"Total power generated ({total_power_generated} MW) is less than total consumer demand ({total_power_demand} MW).")
    

    consumer_demand_map = {consumer["id"]: consumer["demand"] for consumer in consumers}
    for consumer in consumers:
        consumer_id = consumer["id"]
        demand = consumer["demand"]

        incoming_lines = [line for line in transmission_lines if line["to"] == consumer_id]
        if not incoming_lines:
            transmission_capacity_passed = False
            transmission_capacity_errors.append(f"No transmission lines lead to consumer '{consumer_id}'.")
            continue

        if not any(line["max_capacity"] >= demand for line in incoming_lines):
            transmission_capacity_passed = False
            transmission_capacity_errors.append(f"Consumer '{consumer_id}' has a demand of {demand} MW, which exceeds the capacity of all incoming transmission lines.")
    
    if transmission_capacity_passed:
        verification_results["transmission_capacity_compliance"]["passed"] = True
    else:
        verification_results["transmission_capacity_compliance"]["message"] = " ".join(transmission_capacity_errors)
        add_error("Transmission Capacity Compliance Check Failed.")


    demand_fulfillment_passed = True
    demand_fulfillment_errors = []
    if total_power_generated < total_power_demand:
        demand_fulfillment_passed = False
        demand_fulfillment_errors.append(f"Total power generated ({total_power_generated} MW) is less than total consumer demand ({total_power_demand} MW).")
    if demand_fulfillment_passed:
        verification_results["demand_fulfillment"]["passed"] = True
    else:
        verification_results["demand_fulfillment"]["message"] = " ".join(demand_fulfillment_errors)
        add_error("Demand Fulfillment Check Failed.")

    transmission_time_passed = True
    transmission_time_errors = []
    for line in transmission_lines:
        base_time = line["base_time"]
        dynamic_adjust = line["dynamic_adjust"]
        max_adjust = max(dynamic_adjust.values()) if dynamic_adjust else 0
        total_time = base_time + max_adjust
        if total_time > 1440:  # 24 hours in minutes
            transmission_time_passed = False
            transmission_time_errors.append(f"Transmission Line from '{line['from']}' to '{line['to']}' has a total transmission time of {total_time} minutes, exceeding 24 hours.")
    if transmission_time_passed:
        verification_results["transmission_time_compliance"]["passed"] = True
    else:
        verification_results["transmission_time_compliance"]["message"] = " ".join(transmission_time_errors)
        add_error("Transmission Time Compliance Check Failed.")

    overall_passed = all([
        verification_results["output_structure"]["passed"],
        verification_results["storage_capacity_compliance"]["passed"],
        verification_results["transmission_capacity_compliance"]["passed"],
        verification_results["demand_fulfillment"]["passed"],
        verification_results["transmission_time_compliance"]["passed"]
    ])
    verification_results["overall_passed"] = overall_passed

    return overall_passed, verification_results

# PASTE HERE!


# Execute verification
verification_output = verify_electric_power_distribution_network(power_plants, substations, consumers, transmission_lines)
print("Verification Output:", verification_output)


In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o1mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "electricNetwork"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_run{run_number}_output.json")


verification_json = {
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)